# Graph recommendation

In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
from typing import Iterator, Tuple
import re
import pickle
import _pickle as cPickle

from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str
import numpy as np
import pandas as pd
import dgl
from dgl.sampling import sample_neighbors, select_topk
from dgl import load_graphs, save_graphs, DGLHeteroGraph
from dgl.data.heterograph_serialize import HeteroGraphData
from typing import Any, Dict, List, Tuple
import torch

import recommender_gnn.pipelines.santander_preprocessing.nodes
from recommender_gnn.extras.datasets.chunks_dataset import (
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
dataset = "santander"

In [ ]:
transactions_graph_path = f"{dataset}.transactions_graph"
transactions_mapped_path = f"{dataset}_transactions_mapped"
users_mapping_path = f"{dataset}_users_mapping"
items_mapping_path = f"{dataset}_items_mapping"
neg_transactions_path = f"{dataset}_dgsr_negative_transactions_samples"

In [ ]:
transactions_mapped = _concat_chunks(context.catalog.load(transactions_mapped_path))
neg_transactions = context.catalog.load(neg_transactions_path)

In [ ]:
from ast import literal_eval

In [ ]:
neg_transactions[0]

# DGL graph loading

In [ ]:
def save_graphs_python(save_filepath: str, graph: dgl.DGLGraph, graph_dict: Dict) -> None:
    """Save heterographs into file using only Python functions instead of dgl C implementation"""
    if graph_dict is None:
        graph_dict = {}
    if isinstance(graph, DGLHeteroGraph):
        graph = [graph]
        graph_dict = [graph_dict]
    assert all(
        [type(g) == DGLHeteroGraph for g in graph]
    ), "Invalid DGLHeteroGraph in graph argument"
    gdata_list = [
        [g, graph_dict[i]] for i, g in enumerate(graph)
    ]
    with open(save_filepath, "wb") as file:
        pickle.dump(gdata_list, file)

In [ ]:
graph_path = "/home/michal/projects/gid-ml-framework/data/05_model_input/santander/santander_dgsr_kaggle_grm_predict_graphs/20580/20580_2.bin"

In [ ]:
with open(graph_path, 'rb') as f:
    gdata_list = pickle.load(f)

In [ ]:
gdata_list[0][1]

In [ ]:
gdata_list = load_graphs(graph_path)

In [ ]:
graph = gdata_list[0][0]
graph_dict = gdata_list[1]

In [ ]:
type(gdata_list[0][0]) == DGLHeteroGraph

In [ ]:
save_filepath = "/home/michal/projects/data_temp/4_1.bin"

In [ ]:
save_graphs_python(save_filepath, graph, graph_dict)

In [ ]:
with open(save_filepath, "wb") as file:
        pickle.dump(gdata_list, file)